# **SAT Solving**

## Informações sobre o aluno

**Nome**: Rui Filipe Pimenta Armada<br>
**Número**: PG50737<br>
**Curso**: Mestrado em Engenharia Informática<br>

In [83]:
pip install python-sat[pblib,aiger]

zsh:1: no matches found: python-sat[pblib,aiger]
Note: you may need to restart the kernel to use updated packages.


In [84]:
from pysat.solvers import Minisat22

## Pergunta 1

In [85]:

# DICIONÁRIO DAS VARIÁVEIS PROPOSICIONAIS
x = {
    'CPU1': 1,
    'CPU2': 2,
    'RAM1': 3,
    'RAM2': 4,
    'MB1': 5,
    'MB2': 6,
    'PG1': 7,
    'PG2': 8,
    'PG3': 9,
    'MON1': 10,
    'MON2': 11,
    'MON3': 12
}


### Formulas proposicionais

1. Cada computador tem que obrigatóriamente uma única motherboard, um único CPU, uma única placa gráfica e uma única memória RAM.

    * Motherboard

        ``` 
            (MB1 ∨ MB2) ∧ (¬MB1 ∨ ¬MB2)
        ```
    * CPU
        ```
            (CPU1 ∨ CPU2) ∧ (¬CPU1 ∨ ¬CPU2)
        ```

    * Placa Gráfica
        ```
            (PG1 ∨ PG2 ∨ PG3) ∧ (¬PG1 ∨ ¬PG2) ∧ (¬PG1 ∨ ¬PG3) ∧ (¬PG2 ∨ ¬PG3)
        ```

    * RAM
        ```
            (RAM1 ∨ RAM2) ∧ (¬RAM1 ∨ ¬RAM2)
        ```


Relativamente à restrição dos monitores, visto que se pode ou nao pode ter monitores, a fórmula será uma tautologia, pelo que irá sempre adquirir o valor Verdadeiro.

2. A motherboard MB1 quando combinada com a placa gráfica PG1, obriga à utilização da RAM1.

``` 
    MB1 ∧ PG1 -> RAM1
        transformando em CNF:
            ¬(MB1 ∧ PG1) ∨ (RAM1)
        concluindo:
            ¬MB1 ∨ ¬PG1 ∨ RAM1
```  

3. A placa gráfica PG1 precisa do CPU1, excepto quando combinada com uma memória RAM2

```
    PG1 -> (CPU1 ∨ RAM2)
        transformando diretamente em CNF:
            ¬PG1 ∨ CPU1 ∨ RAM2
```

4. O CPU2 só pode ser instalado na motherboard MB2

```
    CPU2 -> MB2
        transformando diretamente em CNF:
            ¬ CPU2 ∨ MB2
```

5. O monitor MON1 para poder funcionar precisa da placa gráfica PG1 e da memória RAM2.

```
    MON1 -> PG1 ∧ RAM2
        transformando em CNF:
            ¬MON1 ∨ (PG1 ∧ RAM2)
        concluindo:
            (¬MON1 ∨ PG1) ∧ (¬MON1 ∨ RAM2)
```

6. O monitor MON2 precisa da memória RAM2 para poder trabalhar com a placa gráfica PG3

``` 
    (MON2 ∧ PG3) -> RAM2
        transformando em CNF:
            ¬(MON2 ∧ PG3) ∨ RAM2
        concluindo:
            ¬MON2 ∨ ¬PG3 ∨ RAM2 
```

## Pergunta 2

In [86]:
s = Minisat22() #cria o solver

# RESTRIÇÃO 1
s.add_clause([x['MB1'], x['MB2']])
s.add_clause([-x['MB1'], -x['MB2']])
s.add_clause([x['CPU1'], x['CPU2']])
s.add_clause([-x['CPU1'], -x['CPU2']])
s.add_clause([x['PG1'], x['PG2'], x['PG3']])
s.add_clause([-x['PG1'], -x['PG2']])
s.add_clause([-x['PG1'], -x['PG3']])
s.add_clause([-x['PG2'], -x['PG3']])
s.add_clause([x['RAM1'], x['RAM2']])
s.add_clause([-x['RAM1'], -x['RAM2']])
# RESTRIÇÃO 2
s.add_clause([-x['MB1'], -x['PG1'], x['RAM1']])
# RESTRIÇÃO 3
s.add_clause([-x['PG1'], x['CPU1'], x['RAM2']])
# RESTRIÇÃO 4
s.add_clause([-x['CPU2'], x['MB2']])
# RESTRIÇÃO 5
s.add_clause([-x['MON1'], x['PG1']])
s.add_clause([-x['MON1'],x['RAM2']])
# RESTRIÇÃO 6
s.add_clause([-x['MON2'], -x['PG3'], x['RAM2']])

if s.solve():      
    print("SAT")
    print(s.get_model())
else:
    print("UNSAT")
    
s.delete()

SAT
[1, -2, 3, -4, 5, -6, 7, -8, -9, -10, -11]


Como se pode verificar pelo *output* do *SAT solver*, o conjunto de fórmulas utilizado é consistente. Uma resposta válida para o problema é um computador com a seguinte configuração:

* 1 CPU1
* 1 RAM1
* 1 MB1
* 1 PG1

## Pergunta 3

#### **a)** O monitor MON1 só poderá ser usado com uma motherboard MB1 ?
  * De forma a conseguir responder à questão é necessário inserir clausulas em que o valor lógico de MON1 seja verdadeiro e o de MB1 seja falso. 
  * Se for possível obter um modelo pode-se usar o MON1 com qualquer outra motherboard, caso contrário, apenas pode ser usada a MB1.
  

In [87]:
s = Minisat22() #cria o solver

# RESTRIÇÃO 1
s.add_clause([x['MB1'], x['MB2']])
s.add_clause([-x['MB1'], -x['MB2']])
s.add_clause([x['CPU1'], x['CPU2']])
s.add_clause([-x['CPU1'], -x['CPU2']])
s.add_clause([x['PG1'], x['PG2'], x['PG3']])
s.add_clause([-x['PG1'], -x['PG2']])
s.add_clause([-x['PG1'], -x['PG3']])
s.add_clause([-x['PG2'], -x['PG3']])
s.add_clause([x['RAM1'], x['RAM2']])
s.add_clause([-x['RAM1'], -x['RAM2']])
# RESTRIÇÃO 2
s.add_clause([-x['MB1'], -x['PG1'], x['RAM1']])
# RESTRIÇÃO 3
s.add_clause([-x['PG1'], x['CPU1'], x['RAM2']])
# RESTRIÇÃO 4
s.add_clause([-x['CPU2'], x['MB2']])
# RESTRIÇÃO 5
s.add_clause([-x['MON1'], x['PG1']])
s.add_clause([-x['MON1'],x['RAM2']])
# RESTRIÇÃO 6
s.add_clause([-x['MON2'], -x['PG3'], x['RAM2']])
# CLAUSULAS INSERIDAS
s.add_clause([x['MON1']])
s.add_clause([-x['MB1']])


if s.solve():      
    print("SAT")
    print(s.get_model())
else:
    print("UNSAT")
    
s.delete()

SAT
[1, -2, -3, 4, -5, 6, 7, -8, -9, 10, -11]


Como é possivel obter um modelo, pode-se verificar que o MON1 pode ser usado com qualquer outra motherboard. Uma resposta válida para o problema é um computador com a seguinte configuração:

* 1 CPU1
* 1 RAM2
* 1 MB2
* 1 PG1
* 1 MON1

#### **b)** Um cliente pode personalizar o seu computador da seguinte forma: uma motherboard MB1, o CPU1, a placa gráfica PG2 e a memória RAM1 ?

* De forma a conseguir responder à questão basta inserir clausulas de forma a descrever a configuração do computador pretendido.
* Se for possível obter um modelo idêntico como resultado signidica que a configuração é possível, caso contr+ario, é uma configuração impossível.

In [88]:
s = Minisat22() #cria o solver

# RESTRIÇÃO 1
s.add_clause([x['MB1'], x['MB2']])
s.add_clause([-x['MB1'], -x['MB2']])
s.add_clause([x['CPU1'], x['CPU2']])
s.add_clause([-x['CPU1'], -x['CPU2']])
s.add_clause([x['PG1'], x['PG2'], x['PG3']])
s.add_clause([-x['PG1'], -x['PG2']])
s.add_clause([-x['PG1'], -x['PG3']])
s.add_clause([-x['PG2'], -x['PG3']])
s.add_clause([x['RAM1'], x['RAM2']])
s.add_clause([-x['RAM1'], -x['RAM2']])
# RESTRIÇÃO 2
s.add_clause([-x['MB1'], -x['PG1'], x['RAM1']])
# RESTRIÇÃO 3
s.add_clause([-x['PG1'], x['CPU1'], x['RAM2']])
# RESTRIÇÃO 4
s.add_clause([-x['CPU2'], x['MB2']])
# RESTRIÇÃO 5
s.add_clause([-x['MON1'], x['PG1']])
s.add_clause([-x['MON1'],x['RAM2']])
# RESTRIÇÃO 6
s.add_clause([-x['MON2'], -x['PG3'], x['RAM2']])
# CLAUSULAS INSERIDAS
s.add_clause([x['MB1']])
s.add_clause([x['CPU1']])
s.add_clause([x['PG2']])
s.add_clause([x['RAM1']])

if s.solve():      
    print("SAT")
    print(s.get_model())
else:
    print("UNSAT")
    
s.delete()

SAT
[1, -2, 3, -4, 5, -6, -7, 8, -9, -10, -11]


Portanto, a solução apresentada pelo *SAT solver* é a seguinte:

* 1 CPU1
* 1 RAM1
* 1 MB1
* 1 PG2

Logo, pode observar-se que a solução obtida é idêntica à configuração colocada pela questão, logo pode-se confirmar que o cliente pode personalizar um computador com as componentes propostas

#### **c)** É possivel combinar a motherboard MB2, a placa gráfica PG3 e a RAM1 num mesmo computador ?

* De forma a conseguir responder à questão é necessário inserir clausulas que indiquem as componentes que se pretendem usar.
* Se for possível obter um modelo pode-se confirmar que é possível combinar as componentes desejadas, caso contrário, não é possivel configuar o computador de tal forma.

In [89]:
s = Minisat22() #cria o solver

# RESTRIÇÃO 1
s.add_clause([x['MB1'], x['MB2']])
s.add_clause([-x['MB1'], -x['MB2']])
s.add_clause([x['CPU1'], x['CPU2']])
s.add_clause([-x['CPU1'], -x['CPU2']])
s.add_clause([x['PG1'], x['PG2'], x['PG3']])
s.add_clause([-x['PG1'], -x['PG2']])
s.add_clause([-x['PG1'], -x['PG3']])
s.add_clause([-x['PG2'], -x['PG3']])
s.add_clause([x['RAM1'], x['RAM2']])
s.add_clause([-x['RAM1'], -x['RAM2']])
# RESTRIÇÃO 2
s.add_clause([-x['MB1'], -x['PG1'], x['RAM1']])
# RESTRIÇÃO 3
s.add_clause([-x['PG1'], x['CPU1'], x['RAM2']])
# RESTRIÇÃO 4
s.add_clause([-x['CPU2'], x['MB2']])
# RESTRIÇÃO 5
s.add_clause([-x['MON1'], x['PG1']])
s.add_clause([-x['MON1'],x['RAM2']])
# RESTRIÇÃO 6
s.add_clause([-x['MON2'], -x['PG3'], x['RAM2']])
# CLAUSULAS INSERIDAS
s.add_clause([x['MB2']])
s.add_clause([x['PG3']])
s.add_clause([x['RAM1']])


if s.solve():      
    print("SAT")
    print(s.get_model())
else:
    print("UNSAT")
    
s.delete()

SAT
[1, -2, 3, -4, -5, 6, -7, -8, 9, -10, -11]


Portanto, a solução apresentada pelo *SAT solver* é a seguinte:

* 1 CPU1
* 1 RAM1
* 1 MB1
* 1 PG2

Logo, como é possível obter um modelo, pode-se afirmar que as componentes indicadas podem ser utilizadas em conjunto.

#### **d)** Para combinarmos a placa gráfica PG2 e a RAM1 temos que usar o CPU2 ?

* De forma a conseguir responder à questão é necessário inserir clausulas que indiquem as que se pretende utilizar a PG2 e a RAM1 mas que nao se pretende usar o CPU2.
* Se for possível obter um modelo pode-se confirmar que não é necessário utilizar o CPU2 para combinar a PG2 e a RAM1.

In [90]:
s = Minisat22() #cria o solver

# RESTRIÇÃO 1
s.add_clause([x['MB1'], x['MB2']])
s.add_clause([-x['MB1'], -x['MB2']])
s.add_clause([x['CPU1'], x['CPU2']])
s.add_clause([-x['CPU1'], -x['CPU2']])
s.add_clause([x['PG1'], x['PG2'], x['PG3']])
s.add_clause([-x['PG1'], -x['PG2']])
s.add_clause([-x['PG1'], -x['PG3']])
s.add_clause([-x['PG2'], -x['PG3']])
s.add_clause([x['RAM1'], x['RAM2']])
s.add_clause([-x['RAM1'], -x['RAM2']])
# RESTRIÇÃO 2
s.add_clause([-x['MB1'], -x['PG1'], x['RAM1']])
# RESTRIÇÃO 3
s.add_clause([-x['PG1'], x['CPU1'], x['RAM2']])
# RESTRIÇÃO 4
s.add_clause([-x['CPU2'], x['MB2']])
# RESTRIÇÃO 5
s.add_clause([-x['MON1'], x['PG1']])
s.add_clause([-x['MON1'],x['RAM2']])
# RESTRIÇÃO 6
s.add_clause([-x['MON2'], -x['PG3'], x['RAM2']])
# CLAUSULAS INSERIDAS
s.add_clause([x['PG2']])
s.add_clause([x['RAM1']])
s.add_clause([-x['CPU2']])


if s.solve():      
    print("SAT")
    print(s.get_model())
else:
    print("UNSAT")
    
s.delete()

SAT
[1, -2, 3, -4, 5, -6, -7, 8, -9, -10, -11]


Portanto, a solução apresentada pelo *SAT solver* é a seguinte:

* 1 CPU1
* 1 RAM1
* 1 MB1
* 1 PG2

Logo, como é possível obter um modelo, pode-se concluir que não é necessário utilizar o CPU2 para combinar a PG2 e a RAM1.

<div align="center">
  <sub>Métodos Formais em Engenharia de Software 22/23</sub>
</div>